In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Imports

In [4]:
!pip install pytorch-lightning
!pip install tables==3.6.1

In [5]:
import pandas as pd
import numpy as np
import random
import os
import pickle

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from enum import Enum
import torch.nn.functional as F

import pytorch_lightning as pl

seed = 0

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

# Hyper-params

In [6]:
data_dir = '/gdrive/MyDrive/BTP/Data/BeamSoup'
lidar_dir = '/gdrive/MyDrive/BTP/Data/BeamSoup/lidar_data'
BATCH_SIZE = 32
n_worker = 2
save_dir = '/gdrive/MyDrive/BTP/Models'
os.makedirs(save_dir,exist_ok=True)

# Loading Data

In [7]:
# coord_train = pd.read_hdf(os.path.join(data_dir,'coords_labels.h5'),key='train')
# train_pos = coord_train[['X', 'Y', 'Z', 'LOS']].to_numpy()
# Y = np.array(coord_train[['Labels']].values.tolist())
# Y = Y.reshape((11194,256))
# y_train = np.argmax((Y/(Y.sum(axis=1).reshape((Y.shape[0],1)))),axis=1)

lidar_train = np.load(os.path.join(lidar_dir,'lidar_008.npz'))['input']

In [ ]:
with open(data_dir+'/lidar_008.txt','wb') as a:
    pickle.dump(lidar_train,a)

In [10]:
print(y_train.shape)
lidar_train.shape

(11194,)


(11194, 180, 330, 10)

In [8]:
coord_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11194 entries, 0 to 11193
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       11194 non-null  float64
 1   Y       11194 non-null  float64
 2   Z       11194 non-null  float64
 3   LOS     11194 non-null  float64
 4   Labels  11194 non-null  object 
dtypes: float64(4), object(1)
memory usage: 524.7+ KB


In [9]:
# coord_val = pd.read_hdf(os.path.join(data_dir,'coords_labels.h5'),key='val')
# val_pos = coord_val[['X', 'Y', 'Z', 'LOS']].to_numpy()
# Y = np.array(coord_val[['Labels']].values.tolist())
# Y = Y.reshape((Y.shape[0],256))
# y_val = Y/(Y.sum(axis=1).reshape((Y.shape[0],1)))

# lidar_val = np.load(os.path.join(lidar_dir,'lidar_009.npz'))['input']

# Dataset and dataloader

In [22]:
class custom_dataset(Dataset):
    def __init__(self,lidar:np.array,label:np.array,coord:np.array):
        self.coord = coord
        self.lidar = lidar
        self.label = label
    
    def __getitem__(self,idx):
        sample = dict()

        sample['coord'] = self.coord[idx,:3]
        sample['LOS'] = self.coord[idx,3]
        sample['lidar'] = self.lidar[idx].reshape((10,180,330))
        sample['label'] = self.label[idx]

        return sample_data

    def __len__(self):
        return self.label.shape[0]


In [23]:
train_dataset = custom_dataset(lidar_train,y_train,train_pos)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = n_worker,
    drop_last = True,
    shuffle = True
    )

In [18]:
# dev_dataset = custom_dataset(lidar_val,y_val,val_pos)
# dev_loader = DataLoader(
#     dev_dataset,
#     batch_size=BATCH_SIZE,
#     pin_memory=True,
#     num_workers = n_worker,
#     drop_last = True,
#     )

# Model Class

In [ ]:
class lidar_net(nn.Module):
    def __init__(self,lr):
        super.__init__()
        self.lr = lr
        